In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# THe Brain Tumor Detection

In [2]:
import pandas as pd
import numpy as np
import re
import glob
from PIL import Image

In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15829, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 15829 (delta 21), reused 35 (delta 16), pack-reused 15768
Receiving objects: 100% (15829/15829), 14.65 MiB | 22.29 MiB/s, done.
Resolving deltas: 100% (10833/10833), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.0/608.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00


In [4]:
import torch
from yolov5 import utils

display = utils.notebook_init()

YOLOv5 🚀 v7.0-196-gacdf73b Python-3.10.6 torch-2.0.1+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.3/107.7 GB disk)


In [5]:
# Train YOLOv5m
!python train.py --img 640 --batch 16 --epochs 50 --data /content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/01_DATASET/yolo_v5/brain.yaml --weights yolov5s.pt --cache --project /content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/02_RESULTS_YOLO/YOLOv5

### Loading weight for models

In [6]:
model_v5s = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/02_RESULTS_YOLO/YOLOv5/exp/weights/best.pt')

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-196-gacdf73b Python-3.10.6 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [7]:
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [18]:
def prediction(path_to_vla_img):
  img_val = []
  for img in sorted(glob.glob(f"{path_to_vla_img}"), key=numericalSort):
    n= Image.open(img)
    i= np.asarray(n)
    img_val.append(i)

  predict = model_v5s(img_val)

  return predict

In [19]:
predict=prediction("/content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/01_DATASET/yolo_v5/images/val/*")

In [20]:
len(predict)

30

In [21]:
print(predict)

image 1/30: 630x630 (no detections)
image 2/30: 630x630 (no detections)
image 3/30: 225x225 (no detections)
image 4/30: 250x201 (no detections)
image 5/30: 225x225 (no detections)
image 6/30: 192x192 (no detections)
image 7/30: 417x428 (no detections)
image 8/30: 251x201 (no detections)
image 9/30: 201x173 (no detections)
image 10/30: 201x173 (no detections)
image 11/30: 168x300 (no detections)
image 12/30: 183x275 (no detections)
image 13/30: 168x300 (no detections)
image 14/30: 197x177 (no detections)
image 15/30: 217x232 1 tumor
image 16/30: 380x310 1 tumor
image 17/30: 347x300 1 tumor
image 18/30: 173x189 1 tumor
image 19/30: 342x273 2 tumors
image 20/30: 256x256 (no detections)
image 21/30: 344x279 (no detections)
image 22/30: 331x272 1 tumor
image 23/30: 331x260 1 tumor
image 24/30: 236x213 2 tumors
image 25/30: 256x197 1 tumor
image 26/30: 251x201 1 tumor
image 27/30: 295x283 1 tumor
image 28/30: 352x281 2 tumors
image 29/30: 960x781 1 tumor
image 30/30: 349x292 1 tumor
Speed: 2

In [22]:
#predict.save()

In [23]:
predict.show()

Output hidden; open in https://colab.research.google.com to view.

In [27]:
def labels(path_to_val_ann):
  pred_lab=[]
  for x in range(len(predict)):
    aa=predict.pandas().xyxy[x]
    if len(aa)==0:
      pred_lab.append(0)
    else:
      pred_lab.append(1)

  txt_img = []
  for x in sorted(glob.glob(f"{path_to_val_ann}"), key=numericalSort):
    lab=x.split('/')[-1]
    txt_img.append(lab.split('.')[0])

  true_lab=[]
  for x in txt_img:
    if 'Y' in x:
      true_lab.append(1)
    else:
      true_lab.append(0)

  return pred_lab, true_lab

In [28]:
pred_lab, true_lab = labels("/content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/01_DATASET/yolo_v5/labels/val/*.txt")

In [29]:
print(true_lab)
print(pred_lab)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [30]:
from sklearn.metrics import classification_report

In [31]:
print(classification_report(true_lab, pred_lab))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90        15
           1       0.93      0.87      0.90        15

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30

